<a href="https://colab.research.google.com/github/invictus125/cs598-final-project/blob/main/intraoperative_hypotension.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Reproduction of:
## Predicting intraoperative hypotension using deep learning with waveforms of arterial blood pressure, electroencephalogram, and electrocardiogram

Original paper by: Yong-Yeon Jo, Jong-Hwan Jang, Joon-myoung Kwon, Hyung-Chul Lee, Chul-Woo Jung, Seonjeong Byun, Han‐Gil Jeong

Reproduction project authored by
* Mark Bauer
  * mbauer553
  * markab5@illinois.edu
* Ryan David
  * victheone
  * invictus125
  * radavid2@illinois.edu

> Note that this project uses <b>VitalDB, an open biosignal dataset.  All users must agree to the Data Use Agreement below.</b>  If after reviewing the agreement you do not comply, please do not read on and close this window.
[Data Use Agreement](https://vitaldb.net/dataset/?query=overview&documentId=13qqajnNZzkN7NZ9aXnaQ-47NWy7kx-a6gbrcEsi-gak&sectionId=h.vcpgs1yemdb5)

## Introduction
Our project is to perform an aproximate reproduction of a paper, which can be found [here](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0272055), on predicting hypotension during surgery from a combination of signals such as mean arterial blood pressure (ABP), electrocardiogram (ECG), and electroencephalogram (EEG) as opposed to ABP alone.  Predicting hypotension is important because it is correlated with many post operation complications, and is actionable.  Please read the original work if you are interested in more detail!

## Scope of reproducibility

//TODO

In [2]:
# Update from github
!git clone https://github.com/invictus125/cs598-final-project
!cd /content/cs598-final-project && git pull
!pip install torcheval
!pip install vitaldb

Cloning into 'cs598-final-project'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 117 (delta 55), reused 64 (delta 17), pack-reused 0
Receiving objects: 100% (117/117), 41.51 KiB | 2.96 MiB/s, done.
Resolving deltas: 100% (55/55), done.
Already up to date.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 7.0 MB/s eta 0:00:00


## Methodology - Data
Data descriptions

In [3]:
import csv
from functools import partial
from io import StringIO
import numpy as np
import vitaldb
import requests
from torch import FloatTensor, BoolTensor

SEGEMENT_LENGTH_SECONDS = 60

ABP_TRACK = 'SNUADC/ART'
ECG_TRACK = 'SNUADC/ECG_II'
EEG_TRACK = 'BIS/EEG1_WAV'

RELEVANT_TRACKS = [
    ABP_TRACK,
    ECG_TRACK,
    EEG_TRACK,
]

def _url_to_reader(url_string):
    response = requests.get(url_string)
    file = StringIO(response.text)
    return csv.DictReader(file, delimiter=',')

def get_unique_vals(key, iterable):
    return set(map(lambda item: item[key], iterable))

def case_filter(case):
    return float(case['age']) >= 18.0 and case['ane_type'] == 'General'

def _case_track_filter(case_id, case_dict):
    track_list = case_dict[case_id]['tracks']
    return (
        ABP_TRACK in track_list and
        ECG_TRACK in track_list and
        EEG_TRACK in track_list
    )

def _get_candidate_cases():
    cases_by_id = {}
    for case in _url_to_reader('https://api.vitaldb.net/cases'):
        if case_filter(case):
            case['tracks'] = {}
            cases_by_id[case['\ufeffcaseid']] = case

    track_list_reader = _url_to_reader('https://api.vitaldb.net/trks')

    for track in track_list_reader:
        case_id = track['caseid']
        if track['tname'] in RELEVANT_TRACKS:
            if cases_by_id.get(case_id):
                cases_by_id[case_id]['tracks'][track['tname']] = track['tid']

    case_track_filter = partial(_case_track_filter, case_dict=cases_by_id)

    return [case_id for case_id in filter(case_track_filter, cases_by_id.keys())], cases_by_id

def validate_abp_segment(segment):
    return (
        not np.isnan(segment).mean() > 0.1 and
        not (segment > 200).any() and
        not (segment < 30).any() and
        not ((np.max(segment) - np.min(segment)) < 30) and
        not (np.abs(np.diff(segment)) > 30).any() # abrupt changes are assumed to be noise
    )

def get_data(
    minutes_ahead,
    abp_and_ecg_sample_rate_per_second=500,
    eeg_sample_rate_per_second=128,
    max_num_samples=None,
    max_num_cases=None,
):
    candidate_case_ids, _ = _get_candidate_cases()
    abps = []
    ecgs = []
    eegs = []
    hypotension_event_bools = []

    abp_data_in_two_seconds = 2 * abp_and_ecg_sample_rate_per_second

    case_count = 0
    for case_id in candidate_case_ids:
        case_num_samples = 0
        case_num_events = 0

        print('Getting track data for case:', case_id)
        case_tracks = vitaldb.load_case(int(case_id), RELEVANT_TRACKS[0:2], 1/abp_and_ecg_sample_rate_per_second)

        abp_track = case_tracks[:,0]
        # ecg_track = case_tracks[:,1]

        # eeg_track = vitaldb.load_case(int(case_id), RELEVANT_TRACKS[2], 1/eeg_sample_rate_per_second).flatten()

        for i in range(
            0,
            len(abp_track) - abp_and_ecg_sample_rate_per_second * (SEGEMENT_LENGTH_SECONDS + (1 + minutes_ahead) * SEGEMENT_LENGTH_SECONDS),
            10 * abp_and_ecg_sample_rate_per_second
        ):
            x_segment = abp_track[i:i + abp_and_ecg_sample_rate_per_second * SEGEMENT_LENGTH_SECONDS]
            y_segment_start = i + abp_and_ecg_sample_rate_per_second * (SEGEMENT_LENGTH_SECONDS + minutes_ahead * SEGEMENT_LENGTH_SECONDS)
            y_segement_end = i + abp_and_ecg_sample_rate_per_second * (SEGEMENT_LENGTH_SECONDS + (minutes_ahead + 1) * SEGEMENT_LENGTH_SECONDS)
            y_segment = abp_track[y_segment_start:y_segement_end]

            if validate_abp_segment(x_segment) and validate_abp_segment(y_segment):
                abps.append(x_segment)

                # 2 second moving average
                y_numerator = np.nancumsum(y_segment, dtype=np.float32)
                y_numerator[abp_data_in_two_seconds:] = y_numerator[abp_data_in_two_seconds:] - y_numerator[:-abp_data_in_two_seconds]
                y_moving_avg = y_numerator[abp_data_in_two_seconds - 1:] / abp_data_in_two_seconds

                is_hypotension_event = np.nanmax(y_moving_avg) < 65
                hypotension_event_bools.append(is_hypotension_event)
                case_num_samples = case_num_samples + 1
                if(is_hypotension_event):
                    case_num_events = case_num_events + 1
                print('Valid sample detected, is hypotension event?: ', is_hypotension_event)
            else:
                print('Invalid sample')

            at_max_samples = len(hypotension_event_bools) == max_num_samples
            if at_max_samples:
                break

        case_count = case_count + 1

        if at_max_samples or case_count == max_num_cases:
            if at_max_samples:
                print('Max samples reached')
            else:
                print('Max cases reached')
            break

    return (
        FloatTensor(np.array(abps)),
        FloatTensor(np.array(ecgs)),
        FloatTensor(np.array(eegs)),
        BoolTensor(np.array(hypotension_event_bools)),
    )



## Methodology - Model
Model descriptions


In [ ]:
# Methodology code here

## Methodology - Training
Computational requirements


In [ ]:
# Training code here

##  Methodology - Evaluation
Metrics descriptions



In [ ]:
# Evaluation code here


### Results
* Results
* Analyses
* Plans


## References
Jo YY, Jang JH, Kwon Jm, Lee HC, Jung CW, et al. (2022) Predicting intraoperative hypotension using deep learning with waveforms of arterial blood pressure, electroencephalogram, and electrocardiogram: Retrospective study. PLOS ONE 17(8): e0272055. https://doi.org/10.1371/journal.pone.0272055

## Acknowledgements
* As mentioned in the introduction, this project leveraged the open [vitaldb dataset](https://vitaldb.net/dataset/), and without it would have been impossible in its current form.
* Significant inspiration was drawn from [vital db examples](https://github.com/vitaldb/examples)